# YOLOv8 Hyperparameter Optimization with Optuna

This notebook provides an interactive version of the optuna_search.py script for optimizing YOLOv8 hyperparameters.

## 1. Install Dependencies (Run First!)

In [1]:
import sys
# Install required packages
!{sys.executable} -m pip install opencv-contrib-python ultralytics optuna matplotlib seaborn

  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl (46.2 MB)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\light\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## 2. Setup and Imports

In [2]:
import os
import yaml
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO
import torch
import numpy as np  # Check numpy version
print(f"Using numpy version: {np.__version__}")

# Clear GPU cache
torch.cuda.empty_cache()

Using numpy version: 1.23.5


## 3. Load Configuration

In [3]:
CONFIG_PATH = "config.yaml"

with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# Display config
print("Configuration:")
for key, value in config.items():
    print(f"{key}: {value}")

Configuration:
ï»¿model: yolo11m.pt
epochs: 2
batch_size: 2
img_size: 640
data_yaml: data/dataset.yaml
test_images_dir: data/images/test
experiment_name: yolov11n_hurtigruten
use_wandb: False
wandb_project: DNV Hackathon
wandb_run_name: yolo11s-experiment-1


## 4. Define Objective Function

In [4]:
def objective(trial):
    """
    Objective function for Optuna hyperparameter optimization.
    """
    # Hyperparameter search space
    lr0 = 0.0010437818703644063
    weight_decay = 4.099532698065393e-05
    box = trial.suggest_float("box", 5.0, 10.0)
    cls = trial.suggest_float("cls", 0.1, 1.0)
    dfl = trial.suggest_float("dfl", 1.0, 2.0)
    pose = trial.suggest_float("pose", 8.0, 15.0)

    # Initialize YOLO model
    model = YOLO("yolo11m.pt")

    # Train with hyperparameters
    results = model.train(
        data=config["data_yaml"],
        epochs=config["epochs"],
        imgsz=config["img_size"],
        batch=config["batch_size"],
        name=f"optuna_trial_{trial.number}",
        plots=False,
        amp=True,
        lr0=lr0,
        weight_decay=weight_decay,
        cos_lr=True,
        box=box,
        cls=cls,
        dfl=dfl,
        pose=pose,
        fraction=0.3
    )

    # Get validation mAP50-95 score
    val_metrics = model.val(data=config["data_yaml"], split="val", imgsz=config["img_size"])
    return float(val_metrics.box.map)  # Optuna maximizes this

## 5. Run Optimization

In [5]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# Get best trial results
best_trial = study.best_trial
print(f"Best Trial: {best_trial.number}")
print(f"Best mAP50-95: {best_trial.value}")
print(f"Best Hyperparameters: {best_trial.params}")

[I 2025-04-04 22:57:47,653] A new study created in memory with name: no-name-aaef3de4-d134-4597-8170-24dd053ce1a0


New https://pypi.org/project/ultralytics/8.3.101 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.90  Python-3.11.9 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i7-13620H)
engine\trainer: task=detect, mode=train, model=yolo11m.pt, data=data/dataset.yaml, epochs=2, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=optuna_trial_08, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=0.3, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_

[W 2025-04-04 22:57:48,444] Trial 0 failed with parameters: {'box': 9.482643463047353, 'cls': 0.7014782644387177, 'dfl': 1.3229144586760355, 'pose': 13.67750244420572} because of the following error: ModuleNotFoundError("No module named 'numpy._core'").
Traceback (most recent call last):
  File "C:\Users\light\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\light\AppData\Local\Temp\ipykernel_22108\496121724.py", line 17, in objective
    results = model.train(
              ^^^^^^^^^^^^
  File "C:\Users\light\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ultralytics\engine\model.py", line 791, in train
    self.trainer.train()
  File "C:\Users\light\AppData\Local\Packages\PythonSoftwareFoundation.Pyt

ModuleNotFoundError: No module named 'numpy._core'

## 6. Visualize Results

In [ ]:
# Plot optimization history
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

# Plot parameter importance
fig = optuna.visualization.plot_param_importances(study)
fig.show()

# Plot parallel coordinate
fig = optuna.visualization.plot_parallel_coordinate(study)
fig.show()